In [1]:
# Mount Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Specify working directory.
FOLDERNAME = 'cs231n/project/'
%cd /content/drive/My\ Drive/$FOLDERNAME

# Ensure that the Python interpreter of the Colab VM can load python files from
# within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive
/content/drive/My Drive/cs231n/project


## Load dataset and labels

In [ ]:
import numpy as np
import os

base_dir = '/content/drive/My Drive/cs231n/project/dataset/shaped_data'

train_data = np.load(os.path.join(base_dir, 'train_data.npy'))
val_data = np.load(os.path.join(base_dir, 'val_data.npy'))

train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
val_labels = np.load(os.path.join(base_dir, 'val_labels.npy'))

In [ ]:
print('Training data shape: ', train_data.shape)
print('Training label shape: ', train_labels.shape)
print('Train Label=1 ratio:', np.sum(train_labels ) /train_labels.shape[0])
print('Validation data shape: ', val_data.shape)
print('Validation label shape: ', val_labels.shape)
print('Validation Label=1 ratio:', np.sum(val_labels) / val_labels.shape[0])

## Model 1: CNN and LSTM (basic architecture)

In [4]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
model = models.Sequential()
model.add(TimeDistributed(Conv2D(128, (3, 3), strides=(1,1),activation='relu'),
                                input_shape=(16, 250, 250, 1)))
model.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model.add(TimeDistributed(Conv2D(32, (3, 3), strides=(1,1),activation='relu')))
model.add(TimeDistributed(MaxPooling2D(2,2)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.2))

model.add(LSTM(32, return_sequences=False, dropout=0.2)) # used 32 units

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 16, 248, 248, 128) 1280      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 246, 246, 64)  73792     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 123, 123, 64)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 121, 121, 64)  36928     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 119, 119, 32)  18464     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 59, 59, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 16, 59, 59, 32)    1

In [9]:
import keras
callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc', patience=5),
                  keras.callbacks.ModelCheckpoint(
                      filepath='cnn_lstm_model_v1',
                      monitor='val_loss',
                      save_best_only=True
                  ),
                  keras.callbacks.ReduceLROnPlateau(
                      monitor="val_loss", factor=0.1, patience=3
                  )]

from keras import optimizers

optimizer_new=optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer_new,
                    loss='binary_crossentropy',
                    metrics=['acc'])

In [ ]:
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    batch_size=10, epochs=10,
                    class_weight={0:1,1:4},
                    callbacks=callbacks_list)

Epoch 1/10


In [ ]:
acc=history.history["acc"]
val_acc=history.history["val_acc"]
loss=history.history["loss"]
val_loss=history.history["val_loss"]
epoch_stop = 10
epochs=np.arange(1, epoch_stop+1)

plt.plot(epochs, acc, label = "Train", color = 'darkblue')
plt.plot(epochs, val_acc, label = "Valid", linestyle="--", color = 'darkred')
plt.legend()
plt.title('Train/Valid Accuracy')
plt.show()

plt.plot(epochs, loss, label = "Train", color = 'darkblue')
plt.plot(epochs, val_loss, label = "Valid", linestyle="--", color = 'darkred')
plt.legend()
plt.title('Train/Valid Loss')
plt.show()

In [ ]:
# preds = model.predict_classes(X_test)
# print(classification_report(train_labels, preds))